### Blog Generation Workflow Example
This notebook demonstrates how to use LangGraph and Cohere LLM to generate a blog post in two steps: creating an outline and then generating the full content.

In [ ]:
# Import required libraries and set up environment
from langgraph.graph import StateGraph, START, END
from langchain_cohere import ChatCohere
import os
from dotenv import load_dotenv
from typing import TypedDict

load_dotenv()

In [ ]:
# Initialize Cohere API key and model
cohere_api_key = os.getenv("COHERE_API_KEY")
model = ChatCohere(model="command-a-03-2025")

In [ ]:
# Define the state structure for the blog workflow
class blog_state(TypedDict):
    title: str      # Blog post title
    outline: str   # Generated outline
    content: str   # Generated blog content

In [ ]:
# Node function to create a blog outline from the title
def create_outline(state: blog_state):
    title = state["title"]
    prompt = f"Write an outline for a blog post about {title}"
    outline = model.invoke(prompt).content
    state["outline"] = outline
    return state

In [ ]:
# Node function to create blog content from the title and outline
def create_blog_content(state: blog_state):
    title = state["title"]
    outline = state["outline"]
    prompt = f"Write a blog post about {title} with the following outline: {outline}"
    content = model.invoke(prompt).content
    state["content"] = content
    return state

In [ ]:
# Create and compile the workflow graph for blog generation
graph = StateGraph(blog_state)
graph.add_node("create_outline", create_outline)
graph.add_node("create_blog_content", create_blog_content)
graph.add_edge(START, "create_outline")
graph.add_edge("create_outline", "create_blog_content")
graph.add_edge("create_blog_content", END)
workflow = graph.compile()

In [ ]:
# (Optional) Compile the graph again if needed (already compiled above)
graph.compile()

In [ ]:
# Prepare an initial state and run the workflow to generate a blog post
initial_state = {"title": "The Power of AI"}
final_state = workflow.invoke(initial_state)

In [ ]:
# Print the final state containing title, outline, and content
print(final_state)

In [ ]:
# Print only the blog title
print(final_state["title"])

In [ ]:
# Print the generated outline
print(final_state["outline"])

In [ ]:
# Print the generated blog content
print(final_state["content"])